In [1]:
# test file for test.py

In [2]:
from test import test

In [3]:
from __future__ import print_function

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [4]:
# random seeds
torch.manual_seed(665)
np.random.seed(326)

In [5]:
# simple model
# classical feed forward model with variable number of hidden layers and units per layer
class Classic_MNIST_Net(nn.Module):
    
    def __init__(self,):
        super(Classic_MNIST_Net, self).__init__()
        
        # one layer
        in_feats=28*28
        self.fl = nn.Linear(in_feats, 10)
        
    def forward(self, x):
        # squash the image
        x = x.view(-1, 28*28)
        x = self.fl(x)
        # softmax
        x = F.log_softmax(x, dim=-1)
        return x

In [6]:
# Device: cpu only for reproducibility
device = torch.device('cpu')

In [7]:
# define model
model = Classic_MNIST_Net().to(device)

In [8]:
# load some data (MNIST)
batch_size = 10000
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/home/felix/Research/Adversarial Research/MNIST-dataset', train=False, download=False, 
                   transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))
            ])), 
        batch_size=batch_size, shuffle=False)

In [9]:
# loss function
def loss_func(model, output, target):
    return F.nll_loss(output, target)

In [10]:
# number of correct pred function
def pred_func(output, target):
    pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
    correct = pred.eq(target.view_as(pred)).sum().item()
    return correct

In [11]:
### test 1:
res = test(model, device, test_loader, loss_func, verbose=True, pred_func=pred_func)

Test set - Average loss: 0.0002, Accuracy: 1068/10000 (11%)


In [12]:
# # Expected

# Test set: Average loss: 0.0002, Accuracy: 1068/10000 (11%)

In [13]:
print(res)

{'test_loss': tensor(0.0002), 'test_accuracy': 10.68}
